In [1]:
%load_ext tensorboard
import os 
from helpers_image_classification import *
from load_data import *
import cv2
import random
import itertools
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10000)
from load_data import *
from itertools import chain
from collections import Counter
import sklearn.metrics
import io
import re
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
from collections import Counter
from PIL import ImageStat, Image
import shap

Init Plugin
Init Graph Optimizer
Init Kernel


# Log Prices and Max Pooling

In [2]:
# load paths and labels
img_folder = "data/images_resized"
img_df = pd.read_csv("data/img_paths.csv")
#img_df = img_df[img_df["label"] == 0]
img_df.reset_index(drop=True, inplace = True)

# load label book
label_cat = ["bathroom", "bedroom", "dining", "hallway", "kitchen", "living", "others"]
label = np.arange(7)
label_book = pd.DataFrame({"label": label_cat, "categorical_label": label})

In [3]:
# prices
url_listing = "http://data.insideairbnb.com/ireland/leinster/dublin/2021-11-07/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
urls = listings["listing_url"]
ids = listings["id"]
price = listings["price"]
price = price.str.replace("$","")
price = price.str.replace(",","")
price = price.astype(float)
listings["price"] = price
listings["log_price"] = np.log(price)
listings = listings[listings["price"]<500]


In [4]:
# save relevant columns
price_df = listings[["log_price", "id"]]
# merge image and price data together to get the corresponding price for each image
df = pd.merge(img_df, price_df, on = "id", how = "left")
df.head()

,img_path,id,img_no,label,log_price
0,44077_0.png,44077,0,4.0,4.174387
1,44077_1.png,44077,1,5.0,4.174387
2,44077_2.png,44077,2,1.0,4.174387
3,44077_3.png,44077,3,1.0,4.174387
4,44077_4.png,44077,4,1.0,4.174387


In [5]:
filter = np.any(df.isna(), axis = 1)
df = df[~filter]

# drop "others" as we are only interested in the rooms
filter = df["label"] == 6.0
df = df[~filter]

# use only listings with images for 4 or categories 
bool = []
for id in df["id"]:
    tmp = df[df["id"] == id]
    if len(np.unique(tmp["label"])) >= 4:
        bool.append(True)
    else:
        bool.append(False)
df_new = df[bool]
np.unique(df_new["id"].values).shape
df = df_new
df.shape

(56704, 5)

In [6]:
def data_generator(df):
    ''' Extracts the features from a Resnet50 '''
    counter_overall_dummy = 0
    counter_overall_img = 0

    def input_pipeline(room = 0, df = df):
        ids = []
        features = []
        
        # FILTER DF
        df_room = df[df["label"] == room]
        
        # RESNET
        resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
        resnet_pre = keras.applications.resnet50.preprocess_input
        resnet.trainable = False
        counter_dummy = 0
        counter_img = 0
        for id in tqdm_notebook(np.unique(df["id"])):
            filter = df_room["id"] == id  

            try:
                l = []
                if filter.sum() == 0:
                    dummy_image = np.zeros((1,256,256,3))
                    dummy_image = resnet_pre(dummy_image)
                    dummy_image = resnet(dummy_image)
                    l.append(dummy_image)
                    counter_dummy += 1                
                else:
                    path_id = df_room["img_path"][filter]
                    for p in path_id:
                        img_tmp = plt.imread("data/images_resized/"+p)
                        img_tmp = np.expand_dims(img_tmp, axis = 0)
                        img_tmp = resnet_pre(img_tmp)
                        img_tmp = resnet(img_tmp)
                        l.append(img_tmp)
                        counter_img += 1
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
            except:
                dummy_image = np.zeros((1,256,256,3))
                dummy_image = resnet_pre(dummy_image)
                dummy_image = resnet(dummy_image)
                l.append(dummy_image)
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
                continue
        features = np.squeeze(np.stack(features))
        filter = np.nonzero(features.sum(axis = 0))[0]
        features = features[:,filter]
        print(len(filter), " features are nonzero.")
        features = features.tolist()
        print(counter_dummy, "dummy images were added.")
        return features, ids, counter_dummy, counter_img

    basis_df = df[["id","log_price"]]
    basis_df = basis_df.drop_duplicates()
    
    features = []
    ids = []
    
    for i in tqdm_notebook(np.unique(df["label"])):
        feat_cat, ids_cat, counter_dummy, counter_img = input_pipeline(i,df)
        df_tmp = pd.DataFrame({"features_"+str(i): feat_cat, "id": ids_cat})
        basis_df = pd.merge(basis_df, df_tmp, on = "id", how = "left")
        counter_overall_dummy += counter_dummy
        counter_overall_img += counter_img
    print(counter_overall_dummy)
    print(counter_overall_img)
    return basis_df
    

In [7]:
final_df = data_generator(df)

  0%|          | 0/6 [00:00<?, ?it/s]

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-02-22 14:03:03.366285: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-22 14:03:03.366398: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


  0%|          | 0/4356 [00:00<?, ?it/s]

1416  features are nonzero.
266 dummy images were added.


...

1415  features are nonzero.
2428 dummy images were added.


  0%|          | 0/4356 [00:00<?, ?it/s]

1488  features are nonzero.
189 dummy images were added.


  0%|          | 0/4356 [00:00<?, ?it/s]

1422  features are nonzero.
342 dummy images were added.
5583
56704


In [8]:
# define input of model a tuples
def transform(ds):
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
   # others = tf.unstack(ds["others"])

    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living), y

In [9]:
X_train, X_test, X_val, y_train, y_test, y_val = load_data(drop_id = False)

train_ids = X_train["id"]
test_ids = X_test["id"]
val_ids = X_val["id"]

X_train = pd.merge(train_ids, final_df, on = "id", how = "left")
X_train = X_train.drop(["id","log_price"], axis = 1)
X_train.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]
filter = np.any(X_train.isna(), axis = 1)
X_train = X_train[~filter]
y_train = y_train[~filter]

X_test = pd.merge(test_ids, final_df, on = "id", how = "left")
X_test = X_test.drop(["id","log_price"], axis = 1)
X_test.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]
filter = np.any(X_test.isna(), axis = 1)
X_test = X_test[~filter]
y_test = y_test[~filter]

X_val = pd.merge(val_ids, final_df, on = "id", how = "left")
X_val = X_val.drop(["id","log_price"], axis = 1)
X_val.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]
filter = np.any(X_val.isna(), axis = 1)
X_val = X_val[~filter]
y_val = y_val[~filter]


In [10]:
# count how many training images we got
train_id_final = pd.merge(train_ids, final_df, on = "id", how = "left")["id"]
counter = 0
for i in train_id_final:
    counter += (df["id"] == i).sum()
counter

35785

In [11]:
# define weight constraint that ensures incoming weights for a layer to be greater than 0 and sum up top 1 
from tensorflow.keras import backend
class weight_constr(tf.keras.constraints.Constraint):
  ''' Constrains weight tensors to sum up to 1 and being greater than 0 '''

  def __init__(self):
    self.ref_value = 1

  def __call__(self, w):
    nonneg = w * tf.cast(tf.greater_equal(w, 0.), backend.floatx())
    sum_w = tf.reduce_sum(nonneg)
    nonneg_one = nonneg/sum_w
    return nonneg_one


In [12]:
# setup model
class Img_model(tf.keras.Model):

  def __init__(self, dropout = 0, l2 = 0, nodes1 = 512, nodes2 = 1):
    super().__init__()
    
    # bathroom
    self.bn1_bath = tf.keras.layers.BatchNormalization()
    self.drop1_bath = tf.keras.layers.Dropout(dropout)
    self.dense1_bath = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bath = tf.keras.layers.BatchNormalization()
    self.drop2_bath = tf.keras.layers.Dropout(dropout)
    #self.dense2_bath = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bath = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
 
    # bedroom
    self.bn1_bed = tf.keras.layers.BatchNormalization()
    self.drop1_bed = tf.keras.layers.Dropout(dropout)
    self.dense1_bed = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bed = tf.keras.layers.BatchNormalization()
    self.drop2_bed = tf.keras.layers.Dropout(dropout)
    #self.dense2_bed = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bed = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
       
      # dining
    self.bn1_dining = tf.keras.layers.BatchNormalization()
    self.drop1_dining = tf.keras.layers.Dropout(dropout)
    self.dense1_dining = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_dining = tf.keras.layers.BatchNormalization()
    self.drop2_dining = tf.keras.layers.Dropout(dropout)
    #self.dense2_dining = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_dining = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
   
    # hall
    self.bn1_hall = tf.keras.layers.BatchNormalization()
    self.drop1_hall = tf.keras.layers.Dropout(dropout)
    self.dense1_hall = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_hall = tf.keras.layers.BatchNormalization()
    self.drop2_hall = tf.keras.layers.Dropout(dropout)
    #self.dense2_hall = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_hall = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
      # kitchen
    self.bn1_kitchen = tf.keras.layers.BatchNormalization()
    self.drop1_kitchen = tf.keras.layers.Dropout(dropout)
    self.dense1_kitchen = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_kitchen = tf.keras.layers.BatchNormalization()
    self.drop2_kitchen = tf.keras.layers.Dropout(dropout)
    #self.dense2_kitchen = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_kitchen = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
  
      # livingroom
    self.bn1_living = tf.keras.layers.BatchNormalization()
    self.drop1_living = tf.keras.layers.Dropout(dropout)
    self.dense1_living = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_living = tf.keras.layers.BatchNormalization()
    self.drop2_living = tf.keras.layers.Dropout(dropout)
    #self.dense2_living = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_living = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
      # others
    self.bn1_others = tf.keras.layers.BatchNormalization()
    self.drop1_others = tf.keras.layers.Dropout(dropout)
    self.dense1_others = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_others = tf.keras.layers.BatchNormalization()
    self.drop2_others = tf.keras.layers.Dropout(dropout)
    #self.dense2_others = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_others = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))

    # final prediction
    self.bn_final = tf.keras.layers.BatchNormalization()
    self.drop_final = tf.keras.layers.Dropout(dropout)
    self.dense_final = tf.keras.layers.Dense(1, kernel_constraint =weight_constr())
    
  def call(self, inputs, training = None):
    
    # bathroom
    x = self.bn1_bath(inputs[0])
    x = self.drop1_bath(x)
    x = self.dense1_bath(x)
    x = self.bn2_bath(x)
    x = self.drop2_bath(x)
    bath_out = self.dense2_bath(x)
    
    
    # bedroom
    x = self.bn1_bed(inputs[1])
    x = self.drop1_bed(x)
    x = self.dense1_bed(x)
    x = self.bn2_bed(x)
    x = self.drop2_bed(x)
    bed_out = self.dense2_bed(x)
    
    # diningroom
    x = self.bn1_dining(inputs[2])
    x = self.drop1_dining(x)
    x = self.dense1_dining(x)
    x = self.bn2_dining(x)
    x = self.drop2_dining(x)
    dining_out = self.dense2_dining(x)
    
    # hallroom
    x = self.bn1_hall(inputs[3])
    x = self.drop1_hall(x)
    x = self.dense1_hall(x)
    x = self.bn2_hall(x)
    x = self.drop2_hall(x)
    hall_out = self.dense2_hall(x)
    
    # kitchen
    x = self.bn1_kitchen(inputs[4])
    x = self.drop1_kitchen(x)
    x = self.dense1_kitchen(x)
    x = self.bn2_kitchen(x)
    x = self.drop2_kitchen(x)
    kitchen_out = self.dense2_kitchen(x)
    
    # livingroom
    x = self.bn1_living(inputs[5])
    x = self.drop1_living(x)
    x = self.dense1_living(x)
    x = self.bn2_living(x)
    x = self.drop2_living(x)
    living_out = self.dense2_living(x)

    # join
    out = tf.keras.layers.concatenate([bath_out, bed_out, dining_out, hall_out, kitchen_out, living_out])#, others_out])
    out = self.bn_final(out)
    out = self.drop_final(out)
    return self.dense_final(out)

In [13]:
# define r^2 metric
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

## Grid Search

In [ ]:
# load data
X_train_init, X_test_init, X_val_init, y_train_init, y_test_init, y_val_init = load_data(drop_id = False)

# join train and validation data as we only need the indicies
X = pd.concat((X_train_init,X_val_init)).reset_index(drop = True)
y = pd.concat((y_train_init, y_val_init)).reset_index(drop=True)

# define metrics to track
para_do = []
para_l2 = []
para_nodes = []
best_val_mse = []
best_val_rsq = []
best_train_mse = []
best_train_rsq = []

kf = KFold(n_splits=5)

for do in tqdm_notebook([0,0.2,0.5], desc = "Dropout"): # best 0.2, 0.001, 64 
    for l2 in tqdm_notebook([0,0.01,0.001], desc = "L2-Reg"):
        for nodes in tqdm_notebook([64, 512], desc = "Nodes"):
            best_val_mse_tmp = []
            best_val_rsq_tmp = []     
            best_train_mse_tmp = []
            best_train_rsq_tmp = []    
            for train_idx, val_idx in tqdm_notebook(kf.split(X), total = 5):
                
                # data processing
                X_train = X.loc[train_idx,:]
                X_val = X.loc[val_idx,:]
                X_test = X_test_init

                y_train = y[train_idx]
                y_val = y[val_idx]
                y_test = y_test_init

                train_ids = X_train["id"]
                test_ids = X_test_init["id"]
                val_ids = X_val["id"]

                X_train = pd.merge(train_ids, final_df, on = "id", how = "left")
                X_train = X_train.drop(["id","log_price"], axis = 1)
                X_train.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
                filter = np.any(X_train.isna(), axis = 1).values
                X_train = X_train[~filter]
                y_train = y_train[~filter]

                X_test = pd.merge(test_ids, final_df, on = "id", how = "left")
                X_test = X_test.drop(["id","log_price"], axis = 1)
                X_test.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
                filter = np.any(X_test.isna(), axis = 1).values
                X_test = X_test[~filter]
                y_test = y_test[~filter]

                X_val = pd.merge(val_ids, final_df, on = "id", how = "left")
                X_val = X_val.drop(["id","log_price"], axis = 1)
                X_val.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
                filter = np.any(X_val.isna(), axis = 1).values
                X_val = X_val[~filter]
                y_val = y_val[~filter]
                
                batch_size = 128

                # setup tf datasets
                data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                                "bed": np.squeeze(np.stack(X_train["bed"])),
                                                                "dining": np.squeeze(np.stack(X_train["dining"])),
                                                                "hall": np.squeeze(np.stack(X_train["hall"])),
                                                                "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                                "living": np.squeeze(np.stack(X_train["living"])),
                                                                "price": y_train})
                data_train = data_train.cache()
                data_train = data_train.shuffle(6000, seed = 13)
                train_dataset = data_train.take(len(y_train))
                train_dataset = train_dataset.map(transform)
                train_dataset = train_dataset.batch(batch_size)

                data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                                "bed": np.squeeze(np.stack(X_test["bed"])),
                                                                "dining": np.squeeze(np.stack(X_test["dining"])),
                                                                "hall": np.squeeze(np.stack(X_test["hall"])),
                                                                "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                                "living": np.squeeze(np.stack(X_test["living"])),
                                                                "price": y_test})
                data_test = data_test.cache()
                test_dataset = data_test.take(len(y_test))
                test_dataset = test_dataset.map(transform)
                test_dataset = test_dataset.batch(batch_size)

                data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                                "bed": np.squeeze(np.stack(X_val["bed"])),
                                                                "dining": np.squeeze(np.stack(X_val["dining"])),
                                                                "hall": np.squeeze(np.stack(X_val["hall"])),
                                                                "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                                "living": np.squeeze(np.stack(X_val["living"])),
                                                                "price": y_val})
                data_val = data_val.cache()
                val_dataset = data_val.take(len(y_val))
                val_dataset = val_dataset.map(transform)
                val_dataset = val_dataset.batch(batch_size)
                
                # setup model and train
                model = Img_model(dropout= do, nodes1 = nodes, nodes2 = 1, l2 = l2)
                
                lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)
                
                model.compile(optimizer=keras.optimizers.Adam(lr),
                                loss= "mse", metrics= R_squared)
                tf.random.set_seed(2)
                history = model.fit(train_dataset, epochs = 50, validation_data = val_dataset, verbose = 0)
                best_val_mse_tmp.append(np.nanmin(np.array(history.history["val_loss"])))
                best_val_rsq_tmp.append(np.nanmax(np.array(history.history["val_R_squared"])))
                best_train_mse_tmp.append(np.nanmin(np.array(history.history["loss"])))
                best_train_rsq_tmp.append(np.nanmax(np.array(history.history["R_squared"])))
                                        
            # save mean of metrics across folds                           
            best_val_mse.append(np.nanmean(np.array(best_val_mse_tmp)))
            best_val_rsq.append(np.nanmean(np.array(best_val_rsq_tmp)))
            best_train_mse.append(np.nanmean(np.array(best_train_mse_tmp)))
            best_train_rsq.append(np.nanmean(np.array(best_train_rsq_tmp)))
            
            # save parameter combination
            para_do.append(do)
            para_l2.append(l2)
            para_nodes.append(nodes)
                
                

In [50]:
grid_results = pd.DataFrame({"Dropout": para_do, "L2": para_l2, "Nodes": para_nodes, "Val_MSE": best_val_mse, "Val_Rsq": best_val_rsq, "Train_MSE": best_train_mse, "Train_Rsq": best_train_rsq})
grid_results = pd.read_csv("grid_results_priceimage.csv")
grid_results.drop(grid_results.columns[0], axis = 1, inplace = True)

In [51]:
print(grid_results.to_latex(index = False))

\begin{tabular}{rrrrrrr}
\toprule
 Dropout &    L2 &  Nodes &  Val\_MSE &   Val\_Rsq &  Train\_MSE &  Train\_Rsq \\
\midrule
     0.0 & 0.000 &     64 & 0.387561 &  0.066719 &   0.028726 &   0.928711 \\
     0.0 & 0.000 &    512 & 0.374895 &  0.102516 &   0.022524 &   0.944012 \\
     0.0 & 0.010 &     64 & 0.443970 &  0.109065 &   0.281611 &   0.468693 \\
     0.0 & 0.010 &    512 & 0.463205 &  0.109977 &   0.387082 &   0.280092 \\
     0.0 & 0.001 &     64 & 0.461879 &  0.084419 &   0.158667 &   0.768617 \\
     0.0 & 0.001 &    512 & 0.487444 &  0.091255 &   0.172171 &   0.770218 \\
     0.2 & 0.000 &     64 & 0.354459 &  0.147335 &   0.404355 &   0.012711 \\
     0.2 & 0.000 &    512 & 0.348692 &  0.164735 &   0.345854 &   0.161032 \\
     0.2 & 0.010 &     64 & 0.469838 &  0.095086 &   0.740294 &  -0.547127 \\
     0.2 & 0.010 &    512 & 0.527357 &  0.073910 &   0.749915 &  -0.469468 \\
     0.2 & 0.001 &     64 & 0.463425 &  0.141795 &   0.570263 &  -0.126860 \\
     0.2 & 0.001 

In [ ]:
# visualize grid search results
grid_res = grid_results.groupby(["Dropout","L2","Nodes"]).sum()
grid_res.columns = ["Val MSE", r"Val. R^2", "Train MSE", r"Train R^2"]
grid_nice = grid_res[[r"Val. R^2", r"Train R^2"]].style.background_gradient(cmap ='Greens')\
        .set_properties(**{'font-size': '10px'})
grid_nice

In [141]:
grid_res = grid_results.groupby(["Dropout","L2","Nodes"]).sum()
grid_res.columns = ["Val MSE", r"Val. R^2", "Train MSE", r"Train R^2"]
grid_nice2 = grid_res[["Val MSE", "Train MSE"]].style.background_gradient(cmap ='Greens_r')\
        .set_properties(**{'font-size': '10px'})
grid_nice2

Train again for best parameters to find best fold

In [14]:
for i in np.arange(5):
    tf.random.set_seed(123)
    X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold = i+1,drop_id=False)
    
    train_ids = X_train["id"]
    test_ids = X_test["id"]
    val_ids = X_val["id"]

    X_train = pd.merge(train_ids, final_df, on = "id", how = "left")
    X_train = X_train.drop(["id","log_price"], axis = 1)
    X_train.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
    filter = np.any(X_train.isna(), axis = 1).values
    X_train = X_train[~filter]
    y_train = y_train[~filter]

    X_test = pd.merge(test_ids, final_df, on = "id", how = "left")
    X_test = X_test.drop(["id","log_price"], axis = 1)
    X_test.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
    filter = np.any(X_test.isna(), axis = 1).values
    X_test = X_test[~filter]
    y_test = y_test[~filter]

    X_val = pd.merge(val_ids, final_df, on = "id", how = "left")
    X_val = X_val.drop(["id","log_price"], axis = 1)
    X_val.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
    filter = np.any(X_val.isna(), axis = 1).values
    X_val = X_val[~filter]
    y_val = y_val[~filter]

    batch_size = 128

    data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                    "bed": np.squeeze(np.stack(X_train["bed"])),
                                                    "dining": np.squeeze(np.stack(X_train["dining"])),
                                                    "hall": np.squeeze(np.stack(X_train["hall"])),
                                                    "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                    "living": np.squeeze(np.stack(X_train["living"])),
                                                    "price": y_train})
    data_train = data_train.cache()
    data_train = data_train.shuffle(6000, seed = 13)
    train_dataset = data_train.take(len(y_train))
    train_dataset = train_dataset.map(transform)
    train_dataset = train_dataset.batch(batch_size)

    data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                    "bed": np.squeeze(np.stack(X_test["bed"])),
                                                    "dining": np.squeeze(np.stack(X_test["dining"])),
                                                    "hall": np.squeeze(np.stack(X_test["hall"])),
                                                    "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                    "living": np.squeeze(np.stack(X_test["living"])),
                                                    "price": y_test})
    data_test = data_test.cache()
    test_dataset = data_test.take(len(y_test))
    test_dataset = test_dataset.map(transform)
    test_dataset = test_dataset.batch(batch_size)

    data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                    "bed": np.squeeze(np.stack(X_val["bed"])),
                                                    "dining": np.squeeze(np.stack(X_val["dining"])),
                                                    "hall": np.squeeze(np.stack(X_val["hall"])),
                                                    "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                    "living": np.squeeze(np.stack(X_val["living"])),
                                                    "price": y_val})
    data_val = data_val.cache()
    val_dataset = data_val.take(len(y_val))
    val_dataset = val_dataset.map(transform)
    val_dataset = val_dataset.batch(batch_size)

    tf.random.set_seed(2)
    model = Img_model(dropout= 0.2, nodes1 = 512, nodes2 = 1, l2 = 0)

    lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)

    model.compile(optimizer=keras.optimizers.Adam(lr),
                    loss= "mse", metrics= ["mae", R_squared])
    history = model.fit(train_dataset, epochs = 300, validation_data = val_dataset, verbose = 1)
    print("Val loss: " ,np.nanmin(np.array(history.history["val_loss"])))
    print("Val MAE: " ,np.nanmin(np.array(history.history["val_mae"])))
    print("Val R2: ", np.nanmax(np.array(history.history["val_R_squared"])))
    model.evaluate(test_dataset)
                            

2022-02-22 15:30:38.424325: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-22 15:30:38.435320: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/300


2022-02-22 15:30:40.353517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - ETA: 0s - loss: 19.9002 - mae: 4.3768 - R_squared: -47.4141

2022-02-22 15:30:47.402172: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 10s 337ms/step - loss: 19.9002 - mae: 4.3768 - R_squared: -47.4141 - val_loss: 3.0199 - val_mae: 1.6184 - val_R_squared: -6.7949
Epoch 2/300
22/22 [==============================] - 7s 301ms/step - loss: 16.2567 - mae: 3.9670 - R_squared: -38.2907 - val_loss: 4.4761 - val_mae: 2.0229 - val_R_squared: -10.5338
Epoch 3/300
22/22 [==============================] - 7s 304ms/step - loss: 13.1695 - mae: 3.5651 - R_squared: -31.2449 - val_loss: 4.5945 - val_mae: 2.0523 - val_R_squared: -10.8360
Epoch 4/300
22/22 [==============================] - 7s 315ms/step - loss: 10.6938 - mae: 3.2008 - R_squared: -24.7609 - val_loss: 4.5861 - val_mae: 2.0489 - val_R_squared: -10.8107
Epoch 5/300
22/22 [==============================] - 7s 307ms/step - loss: 8.6311 - mae: 2.8614 - R_squared: -19.8220 - val_loss: 4.5942 - val_mae: 2.0487 - val_R_squared: -10.8275
Epoch 6/300
22/22 [==============================] - 7s 297ms/step - loss: 7.0717 - mae: 2.5727 - R_squ

## Train final model

In [29]:
batch_size = 128

X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold = 2 ,drop_id=False)

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 #"others": np.squeeze(np.stack(X_train["others"])),
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                # "others": np.squeeze(np.stack(X_test["others"])),
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 #"others": np.squeeze(np.stack(X_val["others"])),
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [33]:
# track and save training
logdir = "logs/price/ensemble/final/log_price_atleast4"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = logdir+"/checkpoint", monitor='val_R_squared', save_best_only=True, mode='max')
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = Img_model(dropout= 0.2, nodes1 = 512, nodes2 = 1, l2 = 0)
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.9, staircase=False)
model.compile(optimizer=keras.optimizers.Adam(lr),
                loss= "mse", metrics= R_squared)
model.fit(train_dataset, epochs = 300, validation_data = val_dataset, callbacks = [checkpoint_callback, tensorboard_callback])

2022-02-14 15:57:02.231569: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-02-14 15:57:02.231777: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-02-14 15:57:02.232646: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/300


2022-02-14 15:57:04.285919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 2/24 [=>............................] - ETA: 5s - loss: 21.8686 - R_squared: -58.4945  

2022-02-14 15:57:05.920533: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-02-14 15:57:05.920560: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-02-14 15:57:06.313944: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-02-14 15:57:06.333169: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-02-14 15:57:06.356842: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/price/ensemble/final/log_price_atleast4/train/plugins/profile/2022_02_14_15_57_06
2022-02-14 15:57:06.365010: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/price/ensemble/final/log_price_atleast4/train/plugins/profile/2022_02_14_15_57_06/Air-von-Dominik.fritz.box.trace.json.gz
2022-02-14 15:57:06.392105: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creat

24/24 [==============================] - ETA: 0s - loss: 19.5935 - R_squared: -46.9115

2022-02-14 15:57:12.135552: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 10s 302ms/step - loss: 19.5935 - R_squared: -46.9115 - val_loss: 1.2334 - val_R_squared: -1.9088
INFO:tensorflow:Assets written to: logs/price/ensemble/final/log_price_atleast4/checkpoint/assets
Epoch 2/300
24/24 [==============================] - 7s 301ms/step - loss: 15.7364 - R_squared: -37.6142 - val_loss: 5.4295 - val_R_squared: -11.8965
Epoch 3/300
24/24 [==============================] - 9s 362ms/step - loss: 12.5804 - R_squared: -29.9265 - val_loss: 6.6254 - val_R_squared: -14.7441
Epoch 4/300
24/24 [==============================] - 8s 306ms/step - loss: 9.9052 - R_squared: -23.3496 - val_loss: 5.3545 - val_R_squared: -11.7098
Epoch 5/300
24/24 [==============================] - 7s 291ms/step - loss: 7.8761 - R_squared: -18.4170 - val_loss: 4.3538 - val_R_squared: -9.3241
Epoch 6/300
24/24 [==============================] - 8s 313ms/step - loss: 6.2623 - R_squared: -14.2284 - val_loss: 3.6231 - val_R_squared: -7.5837
Epoch 7/300
24/24 [

In [34]:
# final weights to see the contribution - for last epoch
model.layers[-1].get_weights()[0]

array([[0.18464196],
       [0.18135749],
       [0.12258715],
       [0.13475364],
       [0.21524999],
       [0.16140978]], dtype=float32)

In [36]:
# reload best epoch
ckp = "logs/price/ensemble/final/log_price_atleast4/checkpoint"
model.load_weights(ckp)

2022-02-14 16:46:32.039347: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open logs/price/ensemble/final/log_price_atleast4/checkpoint: Failed precondition: logs/price/ensemble/final/log_price_atleast4/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [37]:
# final weights to see the contribution - for best epoch
model.layers[-1].get_weights()[0]

array([[0.21551664],
       [0.21356234],
       [0.1539262 ],
       [0.17068407],
       [0.24233142],
       [0.00397941]], dtype=float32)

In [45]:
cols= ['weight']
index = ["bath", "bed", "dining", "hall", "kitchen", "living"]

weights_rooms = pd.DataFrame(model.layers[-1].get_weights()[0], index=index, columns=cols)

print(weights_rooms.to_latex())

\begin{tabular}{lr}
\toprule
{} &    weight \\
\midrule
bath    &  0.215517 \\
bed     &  0.213562 \\
dining  &  0.153926 \\
hall    &  0.170684 \\
kitchen &  0.242331 \\
living  &  0.003979 \\
\bottomrule
\end{tabular}



In [42]:
pd.DataFrame(model.layers[-1].get_weights()[0], index=Cols, columns=Index).style.background_gradient(cmap ='Greens')\
        .set_properties(**{'font-size': '10px'})

,weight
bath,0.215517
bed,0.213562
dining,0.153926
hall,0.170684
kitchen,0.242331
living,0.003979


In [52]:
# evaluate test data
model.evaluate(test_dataset)

7/7 [==============================] - 3s 366ms/step - loss: 0.3433 - R_squared: 0.1206


[0.34333011507987976, 0.12056545168161392]